# D.A.T.A. API Query Sample
## All 10-K

This sample shows how to retrieve essential deception data for all 10-K documents in DATAbase. Please see our [docs](https://d-a-t-a.developer.azure-api.net/getting-started) for details of how to extend this simple query.

### 1. Load Configuration
Create a local.settings.json configuration file with the following content, and save it to the same folder as your notebook:

`{
    "DATA": {
        "API": {
            "Path": "{path-to-api-from-documentation}",
            "Key": "{your-api-key-from-developer-portal}"
        }
    }
}`

SECURITY: Take care not to check your local.settings.json file into source control such as Github, where the key could easily be stolen.

In [15]:
import json

local_config_file = "./local.settings.json"

with open(local_config_file) as f:
    config = json.load(f)

api_path = config["DATA"]["API"]["Path"]
api_key = config["DATA"]["API"]["Key"]

## User ID is no longer required for this API. Just use your API key, which is automatically linked to your user account.
##user_id = config["DATA"]["API"]["UserID"]

output_csv_file_path = "./output/"
output_csv_file_name = None

#print(api_path)

### 2. Configure Your GraphQL Client

The DATA v1 API exposes a GraphQL endpoint, giving you the power to filter and select data as required. You can build GQL queries by hand, but it is easier to use a client library. 

In [16]:
from python_graphql_client import GraphqlClient

# Your GraphQL client needs the api path and the api key
graphql_client = GraphqlClient(endpoint=api_path, headers={"Data-Subscription-Key":api_key})


### 3. Set your Query Parameters

You can filter on any database property. Here we're looking to download all the main document data for annual reports for this century. We can also limit the number of responses. Full queries can take some seconds to run, so limiting the responses is useful while you are testing, for a faster result.

In [17]:
# Parameters
start_date = "2000-01-01"
end_date = "2022-12-31"
document_types = '["10-K"]'

# Paging functionality. Set to None to retrieve all. Please page up to 500 at a time for best performance.
take = 500
skip = None

### 3. Build your GraphQL Query

This part requires some understanding of GraphQL, but otherwise should be quite straightforward. See our [docs](https://d-a-t-a.developer.azure-api.net/getting-started) for more details on how to filter and shape data.

In [18]:
# User id has been removed from query
graphql_query = """
query {
  companiespaged (
    orderBy: { path: "companyDataScore.FilingDate", descending: true }
    where: { groupedExpressions: [
      { path: "HasCompanyDataScore", comparison: equal, value: "true" }
      { path: "companyDataScore.FinancialDocumentType", comparison: in, """ + f'value: {document_types}' + """ },
      { path: "companyDataScore.FilingDate", comparison: greaterThanOrEqual, """ + f'value: "{start_date}"' + """ },
      { path: "companyDataScore.FilingDate", comparison: lessThanOrEqual, """ + f'value: "{end_date}"' + """ }
    ]}
    """ + (f'first: {take},' if not take is None else '')  + """
    """ + (f'skip: {skip},' if not skip is None else '')  + """
  ) {
    totalCount
    edges {
      node {
        name,
        primaryTicker {
            name
        },
        primaryIndex {
            name
        },
        primarySector {
            name
        },
        companyDataScore {
            document {
                dataScore,
                words,
                deceptiveFragmentsCount,
                truthfulFragmentsCount,
                shortTermFocus,
                longTermFocus,
                internalFocus,
                externalFocus
            },
            financialUncertainty
            filingDate,
            period {
                year,
                quarter
            }
        }
      }
    }
  }
}
"""

#print(graphql_query)

### 4. Execute your GraphQL Query

For queries with large date ranges and no query_limit set, data can be up to 50MB, and queries can take some time to complete.

In [19]:
graphql_result = await graphql_client.execute_async(query=graphql_query)

try:
    print(graphql_result["data"]["companiespaged"]["totalCount"])
except Exception as ex:
    print(graphql_result)
    print(ex)

63802


### 5. Print the results to a csv file



In [20]:
import pandas as pd
from datetime import datetime

flattened_results = []

for edge in graphql_result["data"]["companiespaged"]["edges"]:
    flattened:dict = dict()
    flattened["Name"] = edge["node"]["name"]
    flattened["Ticker"] = edge["node"]["primaryTicker"]["name"]
    flattened["Index"] = edge["node"]["primaryIndex"]["name"]
    flattened["Sector"] = edge["node"]["primarySector"]["name"]
    flattened["Filing Date"] = str(edge["node"]["companyDataScore"]["filingDate"]).split("T")[0]
    flattened["Fiscal Year"] = edge["node"]["companyDataScore"]["period"]["year"]
    flattened["Fiscal Quarter"] = edge["node"]["companyDataScore"]["period"]["quarter"]
    flattened["Data Score"] = edge["node"]["companyDataScore"]["document"]["dataScore"]
    flattened["Words"] = edge["node"]["companyDataScore"]["document"]["words"]
    flattened["Deceptive Fragments Count"] = edge["node"]["companyDataScore"]["document"]["deceptiveFragmentsCount"]
    flattened["Truthful Fragments Count"] = edge["node"]["companyDataScore"]["document"]["truthfulFragmentsCount"]
    if (edge["node"]["companyDataScore"]["document"]["deceptiveFragmentsCount"] + edge["node"]["companyDataScore"]["document"]["truthfulFragmentsCount"]) > 0:
        flattened["Percentage Deceptive Fragments"] = edge["node"]["companyDataScore"]["document"]["deceptiveFragmentsCount"] / (edge["node"]["companyDataScore"]["document"]["deceptiveFragmentsCount"] + edge["node"]["companyDataScore"]["document"]["truthfulFragmentsCount"])
    else:
        flattened["Percentage Deceptive Fragments"] = None
    flattened["Long Term Focus"] = edge["node"]["companyDataScore"]["document"]["longTermFocus"]
    flattened["Short Term Focus"] = edge["node"]["companyDataScore"]["document"]["shortTermFocus"]
    flattened["Internal Focus"] = edge["node"]["companyDataScore"]["document"]["internalFocus"]
    flattened["External Focus"] = edge["node"]["companyDataScore"]["document"]["externalFocus"]
    flattened["Financial Uncertainty"] = edge["node"]["companyDataScore"]["financialUncertainty"]
    flattened_results.append(flattened)

df = pd.read_json(json.dumps(flattened_results))

file_path = (output_csv_file_path + output_csv_file_name) if not output_csv_file_name is None else output_csv_file_path + "output-" + str(datetime.now().timestamp()) + ".csv"

df.to_csv(file_path, encoding='utf-8', index=False)

print("Wrote " + str(len(flattened_results)) + " to csv")


Wrote 500 to csv
